In [1]:
import utils
from mnist_models import TwoLayerNN, MLP, CNN, MLPBN, ConvNet, LeNet, LeNet5
import torch.nn.functional as F

import torch

m = CNN()
batchSize = 100
loss_fn = lambda x, y: F.nll_loss(torch.log(x + 1e-9), y)
learningRate = 0.001

optimizer = torch.optim.Adam(m.parameters(), lr = learningRate)

num_epochs = 5

utils.train_model(m, loss_fn, batchSize, utils.trainset, utils.valset, optimizer, num_epochs)

# torch.save(m, f'checkpoints/m{i:02}')



PermissionError: [Errno 13] Permission denied: '/mnt/Dataset'

In [ ]:
utils.attack_model(m, loss_fn, batchSize, utils.valset, 400);


In [ ]:
for i in range(20):
    m_ = utils.DeltaEnsemble(m, n_neighb = i)
    m_.eval()
    utils.attack_model(m_, loss_fn, batchSize, utils.valset, 400);

In [ ]:
for i in range(5, 20):
    m_ = utils.DeltaEnsemble(m, n_neighb = i)
    m_.eval()
    utils.attack_model(m_, loss_fn, batchSize, utils.valset, 400);

In [ ]:
import numpy as np

var = np.array([[y[2] for y in x[2:]] for x in double_robustness])
np.save('robustness_dual2.npy', var)

In [ ]:
import matplotlib.pyplot as plt

plt.axes().set_aspect('equal')
plt.scatter(var[:, 0], var[:, 1], s=(var[:, 2]*10)**2*10, c=var[:, 2])


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(var[:, 0], var[:, 1], (var[:, 2]*10)**2*10)

ax.set_xlabel('sys 1')
ax.set_ylabel('sys 2')
ax.set_zlabel('ensem')

plt.show()